In [178]:
import os
import pandas as pd
import numpy as np
import pathlib
import tensorflow as tf
import cv2
import time

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adamax

## Getting Images for Training and Validation 

In [3]:
data = []

for dirname, _, filenames in os.walk('cat_v1/'):
    for filename in filenames:
        breed = os.path.basename(dirname)  # Extract breed from the directory
        filepath = os.path.join(dirname, filename)
        data.append({'filename': filename, 'breed': breed})
cwd = os.getcwd()
df = pd.DataFrame(data)

## Resizing Images for Faster Training

In [4]:
# Code from user gpiosenka on Kaggle

start = time.time()
height = 200
width = 224
working_dir = r''
resized_dir = os.path.join(working_dir, 'resized')
os.makedirs(resized_dir, exist_ok=True)  # Create resized_dir if it doesn't exist
source_dir = r'cat_v1'
classes = [klass for klass in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, klass))]

for klass in classes:
    msg = f'Resizing images for class {klass}                                             '
    print(msg, '\r', end='')    
    classpath = os.path.join(source_dir, klass)
    dest_classpath = os.path.join(resized_dir, klass)
    os.makedirs(dest_classpath, exist_ok=True)  # Create class directories in the resized directory
    
    flist = [f for f in os.listdir(classpath) if os.path.isfile(os.path.join(classpath, f))]

    for f in flist:
        fpath = os.path.join(classpath, f)
        dest_fpath = os.path.join(dest_classpath, f)
        try:
            img = cv2.imread(fpath)        
            img = cv2.resize(img, (height, width))
            cv2.imwrite(dest_fpath, img)  # Save the resized image
        except:
            print('file ', fpath, ' can not be processed by cv2')

end = time.time()
duration = end - start
print('Resizing images took ', duration, ' seconds')


file  cat_v1/maine_coon/2003-4288-2848-dsc-8088-2e700.dsc-8088.htm  can not be processed by cv2
Resizing images took  18.913495779037476  seconds


## Image Data Setup for Keras CNN

In [170]:
data_dir = "resized/"
data_dir = pathlib.Path(data_dir)

resized_dims = (200,224)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=resized_dims, 
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=resized_dims,
  batch_size=batch_size)

Found 952 files belonging to 5 classes.
Using 762 files for training.
Found 952 files belonging to 5 classes.
Using 190 files for validation.


## Build Model and Metrics

In [211]:
def F1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
# copied from Keras

In [172]:
img_shape = (200,224,3)
#we resized our images to 200x224, images are RGB, so 3 colors

class_count = len(df['breed'].unique()) #5

inputs = tf.keras.layers.Input(shape=img_shape)
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs, pooling='max')

# Additional layers
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(base_model.output)
x = Dense(256, kernel_regularizer=regularizers.l2(l=0.016),
          activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu')(x)
x = Dropout(rate=0.4, seed=123)(x)
output = Dense(class_count, activation='softmax')(x)

# Create the model
model = tf.keras.models.Model(inputs=inputs, outputs=output)

lr = 0.001
model.compile(optimizer=Adamax(learning_rate=lr), #using legacy adamax because of keras warning 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', F1_score])

print(f'Created EfficientNetB0 model with initial learning rate set to {lr}')


Created EfficientNetB0 model with initial learning rate set to 0.001


## Setup Model & Callbacks for Model Training

In [212]:
csv_logger = CSVLogger('training.log')

cwd = os.getcwd()
checkpoint_filepath = f'{cwd}/checkpoint.model.keras'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

steps_per_epoch = train_ds.cardinality().numpy()
validation_steps = val_ds.cardinality().numpy()

## Model Training and Results

In [185]:
history = model.fit(train_ds, epochs=48, steps_per_epoch=steps_per_epoch, 
                    validation_data=val_ds, validation_steps=validation_steps,
                    callbacks=[model_checkpoint_callback, csv_logger])

Epoch 1/48
24/24 [==============================] - 40s 2s/step - loss: 5.9224 - accuracy: 0.9764 - F1_score: 0.9826 - val_loss: 6.3364 - val_accuracy: 0.8158 - val_F1_score: 0.9669
Epoch 2/48
24/24 [==============================] - 37s 2s/step - loss: 5.4995 - accuracy: 0.9724 - F1_score: 0.9772 - val_loss: 5.8746 - val_accuracy: 0.8000 - val_F1_score: 1.0060
Epoch 3/48
24/24 [==============================] - 32s 1s/step - loss: 5.0721 - accuracy: 0.9803 - F1_score: 0.9516 - val_loss: 5.4686 - val_accuracy: 0.8105 - val_F1_score: 0.9693
Epoch 4/48
24/24 [==============================] - 33s 1s/step - loss: 4.6911 - accuracy: 0.9882 - F1_score: 0.9434 - val_loss: 5.0886 - val_accuracy: 0.8263 - val_F1_score: 0.9814
Epoch 5/48
24/24 [==============================] - 34s 1s/step - loss: 4.3419 - accuracy: 0.9934 - F1_score: 0.9406 - val_loss: 4.7058 - val_accuracy: 0.8211 - val_F1_score: 0.9567
Epoch 6/48
24/24 [==============================] - 40s 2s/step - loss: 4.0183 - accuracy:

In [217]:
log = pd.read_csv('training.log')
print(f'Peak Validation Accuracy: {max(log.val_accuracy)}\nBest Epoch: {np.argmax(log.val_accuracy)+1}')
# printed epoch output is incorrect because an issue with overwriting my training log, 
# should be epoch 25, code is writted properly and the printed output will be correct for any future runs 

Peak Validation Accuracy: 0.878947377204895
Best Epoch: 27
